In [17]:
import os
import re
import PyPDF2
import nltk
from sentence_transformers import SentenceTransformer, util

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danibacaicoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danibacaicoa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'  
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\danibacaicoa\AppData\Local\Temp\ipykernel_16520\201937189.py:1: SyntaxWarning: invalid escape sequence '\M'
  folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'
c:\Users\danibacaicoa\Desktop\VS projects\Library_manager-1\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
folder

'H:\\Mi unidad\\Biblioteca\\Mathematics\\Analysis'

In [20]:
def extract_text_from_pdfs(pdf_folder):
    pdf_texts = {}
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page_num in range(len(reader.pages)):
                    text += reader.pages[page_num].extract_text()
                    
                pdf_texts[filename] = text
    return pdf_texts

In [21]:
Text = extract_text_from_pdfs(folder)


In [31]:
list(Text.keys())[0]

'[B] 2015 - Theory of Distributions (Georgiev, 2015).pdf'

In [23]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum()]
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = [word for word in tokens if word.lower() not in stopwords]
    return ' '.join(tokens)


In [33]:
first_book_title = list(Text.keys())[0]

preprocess_text(first_book_title)

'B 2015 Theory Distributions Georgiev 2015'

In [34]:
first_book = Text[first_book_title]

preprocess_text(first_book)

'Theory DistributionsTheory DistributionsSvetlin Georgiev Theory Distributions 123Svetlin Georgiev Department Differential EquationsUniversity Soﬁa St Kliment Ohridski Soﬁa Bulgaria ISBN ISBN eBook DOI Library Congress Control Number 2015944329 Springer Cham Heidelberg New York Dordrecht London Springer International P ublishing Switzerland 2015 work subject copyright rights rese rved Publisher whether whole part material concerned speciﬁcally rights tra nslation reprinting reuse illustrations recitation broadcasting reproduction microﬁlms oth er physical way transmission information storage retrieval electronic adaptation com puter software similar dissimilar methodology known hereafter use general descriptive names registered names trademarks service marks etc publication imply even absence speciﬁc statement names exempt relevant protective laws regulations therefore free general use publisher authors editors safe assume advice information book believed true accurate date pub licatio